In [2]:
!pip install sklearn-genetic-opt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.7 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn_genetic import GAFeatureSelectionCV
from sklearn_genetic.space import Categorical
from sklearn_genetic.callbacks import ProgressBar
from sklearn_genetic.plots import plot_fitness_evolution
from sklearn.metrics import make_scorer, accuracy_score

x_train = pd.read_csv('x_train.txt', header=None, sep=' ')
y_train = pd.read_csv('y_train.txt', header=None, sep=' ')
x_test = pd.read_csv('x_test.txt', header=None, sep=' ')
x_train.columns = [f"x{i+1}" for i in x_train.columns]
x_test.columns = [f"x{i+1}" for i in x_test.columns]


In [9]:
initial_features = [
    "x1",
    "x10",
    "x101",
    "x102",
    "x103",
    "x104",
    "x105",
    "x106",
    "x132",
    "x140",
    "x149",
    "x153",
    "x156",
    "x176",
    "x191",
    "x2",
    "x22",
    "x221",
    "x229",
    "x253",
    "x286",
    "x3",
    "x304",
    "x322",
    "x323",
    "x324",
    "x329",
    "x336",
    "x35",
    "x352",
    "x36",
    "x4",
    "x40",
    "x404",
    "x413",
    "x423",
    "x459",
    "x463",
    "x499",
    "x5",
    "x58",
    "x6",
    "x65",
    "x7",
    "x74",
    "x8",
    "x81",
    "x9",
    "x99",
]

x_train = x_train[initial_features]
x_test = x_test[initial_features]
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


In [10]:
import pandas as pd
import numpy as np

def calculate_score(estimator, X, y_true, reward=10, punishment=200):
    number_of_features = X.shape[1]
    number_1s = sum(y_true==1)

    y_pred = estimator.predict_proba(X)

    probabilities = y_pred[:,1]
    top_indices = np.argpartition(probabilities, -number_1s)[-number_1s:]
    labels = np.zeros_like(probabilities, dtype=int)
    labels[top_indices] = 1
    labels = labels.tolist()
    y_pred = labels
    correct_class_1 = sum((np.array(y_pred)==1) & (np.array(list(y_true))==1))
    score = reward * correct_class_1*(1000/number_1s) - punishment * number_of_features
    return score

# Genetic Algorithm part 1

In [ ]:
from functools import partial

params_list = [{'population_size': 20, 'mutation_probability': 0.5, 'crossover_probability': 0.2},
          {'population_size': 25, 'mutation_probability': 0.6, 'crossover_probability': 0.3},
          {'population_size': 15, 'mutation_probability': 0.5, 'crossover_probability': 0.4}]

scoring_list = ['accuracy',
           partial(calculate_score, reward=10, punishment=200),
           partial(calculate_score, reward=10, punishment=100)]

results = []

for params in params_list:
  for scoring in scoring_list:
    estimator = LogisticRegression()
    selector = GAFeatureSelectionCV(
        estimator=estimator,
        cv=3,
        scoring=scoring,
        generations=100,
        elitism=True,
        n_jobs=-1,
        verbose=2,
        keep_top_k=8,
        criteria="max",
        **params
    )
    print(f'Selecting for params: {params} and scoring: {scoring}')
    # Fit the genetic algorithm feature selector
    selector.fit(X_train, np.ravel(y_train))
    best_features = selector.support_

    # Transform the dataset to keep only the best features
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    estimator.fit(X_train_selected, y_train)
    accuracy = estimator.score(X_test_selected, y_test)
    results.append({'params': params, 'scoring': scoring, 'features': best_features, 'test_accuracy': accuracy,
                    'test_score': calculate_score(estimator, X_test_selected, np.ravel(y_test))})

    print("Selected features:", best_features)
    print("Accuracy with selected features:", accuracy)

Selecting for params: {'population_size': 20, 'mutation_probability': 0.5, 'crossover_probability': 0.2} and scoring: accuracy


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.536161	0.00595634 	0.546001   	0.52325    
1  	27    	0.538849	0.00303151 	0.545501   	0.532998   
2  	33    	0.5423  	0.00370771 	0.54825    	0.53675    
3  	27    	0.543462	0.00328699 	0.54825    	0.536997   
4  	29    	0.546037	0.00371835 	0.552253   	0.538997   
5  	26    	0.546537	0.00357078 	0.548999   	0.535499   
6  	33    	0.548212	0.00306159 	0.552751   	0.535748   
7  	28    	0.548987	0.00316812 	0.552751   	0.542249   
8  	31    	0.549062	0.00372068 	0.552751   	0.534246   
9  	33    	0.549225	0.00316745 	0.552749   	0.538253   
10 	30    	0.549161	0.00478134 	0.552749   	0.534      
11 	30    	0.551486	0.00386331 	0.5595     	0.540751   
12 	20    	0.551149	0.00481751 	0.5595     	0.53175    
13 	30    	0.551074	0.00327154 	0.552749   	0.539247   
14 	30    	0.549974	0.00496327 	0.552749   	0.539748   
15 	26    	0.550874	0.00435124 	0.553001   	0.539748   
16 	27    	0.550763	0.00448879 	0.553001   	0.53

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was exp

Selected features: [False False False False False False False  True  True  True  True False
  True False  True  True False  True False  True False  True False  True
 False  True  True  True  True  True False False  True  True  True  True
  True False  True  True  True  True  True False False False  True  True
 False]
Accuracy with selected features: 0.554
Selecting for params: {'population_size': 20, 'mutation_probability': 0.5, 'crossover_probability': 0.2} and scoring: functools.partial(<function calculate_score at 0x78b445b73e20>, reward=10, punishment=200)
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	20    	387.259	574.395    	1768.01    	-492.06    
1  	24    	905.945	471.894    	2087.87    	122.902    
2  	27    	1305.92	431.484    	2092.92    	563.047    
3  	32    	1590.2 	406.044    	2092.92    	563.047    
4  	28    	1873.74	265.876    	2512.93    	1158       
5  	24    	2055.18	369.916    	2512.93    	808.025    
6  	24    	2238.17	292.008    	2512.93    	1558.

...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Selected features: [False  True  True False  True  True False  True False  True False False
 False False  True  True  True  True False  True  True False  True  True
  True  True  True  True  True  True  True  True  True False  True False
 False False False False  True False  True  True False False  True False
  True]
Accuracy with selected features: 0.538
Selecting for params: {'population_size': 15, 'mutation_probability': 0.5, 'crossover_probability': 0.4} and scoring: functools.partial(<function calculate_score at 0x78b445b73e20>, reward=10, punishment=200)
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	15    	530.978	727.315    	1692.93    	-771.888   
1  	26    	1241.31	432.776    	1692.93    	338        
2  	28    	1475.34	438.313    	2337.95    	338        
3  	27    	1736.32	354.289    	2337.95    	917.942    
4  	28    	1989.27	283.4      	2467.91    	1363.02    
5  	27    	2243.25	145.799    	2467.91    	2017.89    
6  	28    	2287.92	237.513    	2733.06    	2017.

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Selected features: [False False  True False False  True False False False False False False
  True False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False  True False
 False]
Accuracy with selected features: 0.496
Selecting for params: {'population_size': 15, 'mutation_probability': 0.5, 'crossover_probability': 0.4} and scoring: functools.partial(<function calculate_score at 0x78b445b73e20>, reward=10, punishment=100)
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	15    	2700.99	361.878    	3283.04    	2183.03    
1  	27    	3105.29	237.39     	3392.92    	2668.01    
2  	27    	3255.68	184.881    	3602.94    	2973.06    
3  	26    	3345.98	172.466    	3602.94    	2823.09    
4  	28    	3431.64	86.4298    	3542.95    	3277.95    
5  	30    	3552.31	118.807    	3737.88    	3392.92    
6  	28    	3687.93	140.077    	3912.91    	3423.

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
results

[{'params': {'population_size': 20,
   'mutation_probability': 0.5,
   'crossover_probability': 0.2},
  'scoring': 'accuracy',
  'features': array([False, False, False, False, False, False, False,  True,  True,
          True,  True, False,  True, False,  True,  True, False,  True,
         False,  True, False,  True, False,  True, False,  True,  True,
          True,  True,  True, False, False,  True,  True,  True,  True,
          True, False,  True,  True,  True,  True,  True, False, False,
         False,  True,  True, False]),
  'test_accuracy': 0.554,
  'test_score': -88.97795591182421},
 {'params': {'population_size': 20,
   'mutation_probability': 0.5,
   'crossover_probability': 0.2},
  'scoring': functools.partial(<function calculate_score at 0x78b445b73e20>, reward=10, punishment=200),
  'features': array([ True,  True, False, False, False, False, False, False, False,
         False, False, False, False, False,  True, False, False, False,
         False, False, False, False,

In [ ]:
pd.DataFrame(results).to_csv("genetic_results_29052024.csv")

# 2nd test

In [43]:
from functools import partial

params_list = []

for population_size in [20, 30, 40]:
  for mutation_probability in [0.6, 0.5, 0.4]:
    for crossover_probability in [0.1, 0.2, 0.3]:
      params_list.append({'population_size': population_size, 'mutation_probability': mutation_probability, 'crossover_probability': crossover_probability})

scoring_list = [partial(calculate_score, reward=10, punishment=200)]

results = []

for params in params_list:
  for scoring in scoring_list:
    estimator = LogisticRegression()
    selector = GAFeatureSelectionCV(
        estimator=estimator,
        cv=3,
        scoring=scoring,
        generations=70,
        elitism=True,
        n_jobs=-1,
        verbose=2,
        keep_top_k=8,
        criteria="max",
        **params
    )
    print(f'Selecting for params: {params} and scoring: {scoring}')
    # Fit the genetic algorithm feature selector
    selector.fit(X_train, np.ravel(y_train))
    best_features = selector.support_

    # Transform the dataset to keep only the best features
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    estimator.fit(X_train_selected, y_train)
    accuracy = estimator.score(X_test_selected, y_test)
    results.append({'params': params, 'scoring': scoring, 'features': best_features, 'test_accuracy': accuracy,
                    'test_score': calculate_score(estimator, X_test_selected, np.ravel(y_test))})

    print("Selected features:", best_features)
    print("Accuracy with selected features:", accuracy)

Selecting for params: {'population_size': 20, 'mutation_probability': 0.6, 'crossover_probability': 0.1} and scoring: functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	20    	542.189	482.209    	1532.94    	-106.77    
1  	29    	772.952	474.518    	1532.94    	-72.0179   
2  	24    	949.444	436.487    	1532.94    	223.115    
3  	27    	1128.73	396.084    	1758.03    	553.03     
4  	24    	1449.48	286.755    	1758.05    	778.025    
5  	30    	1325.45	488.532    	1758.05    	-242.041   
6  	33    	1466.97	373.633    	1758.05    	92.9998    
7  	26    	1669.75	213.759    	2183.13    	1338.02    
8  	27    	1814.75	338.834    	2343.04    	1357.94    
9  	30    	1614.24	563.286    	2343.04    	318.04     
10 	27    	1832.99	446.529    	2343.04    	1012.75    
11 	26    	2022.76	302.623    	2392.82    	1532.94    
12 	30    	2142.77	257.012    	2392.82    	1532.94    
13 	35    	1973.5 	707.674    	2392.82    	-297.088   
14 	27    	2078.95	534.542    	2392.82    	203.042    
15 	33    	2183.64	487.604    	2392.82    	332.965    
16 	27    	2382.34	45.7024    	2392.82    	2183.13    
17 	26    

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Selected features: [ True False False False False False False False False  True False False
  True False False False  True False  True  True False False False  True
  True False False False  True False False  True  True False  True False
 False False False False False False False False False False False  True
 False]
Accuracy with selected features: 0.524
Selecting for params: {'population_size': 20, 'mutation_probability': 0.6, 'crossover_probability': 0.2} and scoring: functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200)
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	20    	425.248	707.665    	2042.83    	-966.958   
1  	30    	906.76 	450.587    	1543.02    	142.982    
2  	31    	1219.49	313.234    	1563.04    	683.19     
3  	32    	1247.99	340.245    	1563.04    	418.042    
4  	34    	1313   	335.044    	1563.04    	442.829    
5  	33    	1399.04	282.005    	1563.04    	833.162    
6  	32    	1397.01	353.046    	1563.04    	237.7

...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Selected features: [False False  True False  True False False False  True False False False
 False False False False False  True False  True  True False False False
  True False False  True False False False False False False False False
 False False False False False False False False False False False False
 False]
Accuracy with selected features: 0.545
Selecting for params: {'population_size': 40, 'mutation_probability': 0.4, 'crossover_probability': 0.2} and scoring: functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200)
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	40    	541.981	633.269    	1907.98    	-651.968   
1  	53    	1107.34	470.858    	2177.83    	142.99     
2  	43    	1316.55	342.798    	1758.07    	383.097    
3  	44    	1590.12	264.36     	2112.97    	1072.92    
4  	50    	1746.22	236.519    	2112.97    	1098       
5  	52    	1872.75	269.836    	2192.79    	623.077    
6  	53    	2017.01	182.841    	2337.75    	1732.

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Selected features: [False False False False False False False False  True False False False
 False False False  True False  True False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False  True False False False False
 False]
Accuracy with selected features: 0.511
Selecting for params: {'population_size': 40, 'mutation_probability': 0.4, 'crossover_probability': 0.3} and scoring: functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200)
gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	40    	498.22 	566.132    	1527.96    	-811.868   
1  	56    	817.833	372.667    	1603.11    	52.8694    
2  	58    	1235.31	386.984    	2332.91    	592.994    
3  	60    	1580.41	367.819    	2492.94    	877.902    
4  	53    	1779.31	363.305    	2492.94    	1052.81    
5  	60    	2118.3 	333.441    	2882.97    	1517.67    
6  	53    	2309.82	279.442    	3103.01    	1622.

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:
results

[{'params': {'population_size': 20,
   'mutation_probability': 0.6,
   'crossover_probability': 0.1},
  'scoring': functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200),
  'features': array([ True, False, False, False, False, False, False, False, False,
          True, False, False,  True, False, False, False,  True, False,
          True,  True, False, False, False,  True,  True, False, False,
         False,  True, False, False,  True,  True, False,  True, False,
         False, False, False, False, False, False, False, False, False,
         False, False,  True, False]),
  'test_accuracy': 0.524,
  'test_score': 2650.501002004007},
 {'params': {'population_size': 20,
   'mutation_probability': 0.6,
   'crossover_probability': 0.2},
  'scoring': functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200),
  'features': array([False, False,  True, False, False, False, False, False, False,
          True,  True, False,

[{'params': {'population_size': 20,
   'mutation_probability': 0.6,
   'crossover_probability': 0.1},
  'scoring': functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200),
  'features': array([ True, False, False, False, False, False, False, False, False,
          True, False, False,  True, False, False, False,  True, False,
          True,  True, False, False, False,  True,  True, False, False,
         False,  True, False, False,  True,  True, False,  True, False,
         False, False, False, False, False, False, False, False, False,
         False, False,  True, False]),
  'test_accuracy': 0.524,
  'test_score': 2650.501002004007},
 {'params': {'population_size': 20,
   'mutation_probability': 0.6,
   'crossover_probability': 0.2},
  'scoring': functools.partial(<function calculate_score at 0x7f256d9bcdc0>, reward=10, punishment=200),
  'features': array([False, False,  True, False, False, False, False, False, False,
          True,  True, False,

In [46]:
pd.DataFrame(results).to_csv("genetic_results_30052024.csv")

In [18]:
df1 = pd.read_csv("genetic_results_29052024.csv")
df2 = pd.read_csv("genetic_results_30052024.csv")

df = pd.concat([df1, df2], axis=0)
best_features = list(df.sort_values(by='test_score', ascending=False)['features'][:10])
best_features

['[False False False False False False False False False False False False\n False False False False False False False False False False False False\n False False False False False False False False False False False  True\n False False False False False False False False False False False False\n False]',
 '[False False False False False False False False False False False False\n False False False False False False False False False False False False\n False False False False False False False False False False False False\n  True False False False False False False False False False False False\n False]',
 '[False False False False False False False False False False False False\n False False False False False False False False False False False False\n False False  True False False  True False False False False  True False\n False False False False False False False False False False False False\n False]',
 '[False False False  True False False False False False False False False\n

In [19]:
for features in best_features:
  tmp = [x for x in features.replace("\n", "").replace("[", "").replace("]", "").split()]
  tmp = [val == 'True' for val in tmp]
  print(np.array(initial_features)[tmp])

['x423']
['x459']
['x329' 'x352' 'x413']
['x102' 'x253' 'x322' 'x7']
['x101' 'x153' 'x324' 'x404']
['x105' 'x153' 'x22' 'x404' 'x423']
['x102' 'x2' 'x36' 'x404' 'x65']
['x153' 'x324' 'x329' 'x413']
['x132' 'x2' 'x221' 'x40' 'x7']
['x101' 'x104' 'x149' 'x221' 'x286' 'x304' 'x423']


In [ ]:
from functools import partial

# Define the estimator
estimator = LogisticRegression()

# Convert custom scoring function to a scorer
# custom_scorer = make_scorer(penalized_score, greater_is_better=True)

# Define the genetic algorithm feature selector
selector = GAFeatureSelectionCV(
    estimator=estimator,
    cv=3,
    scoring=partial(calculate_score, reward=10, punishment=50),
    population_size=20,
    generations=100,
    mutation_probability=0.6,
    crossover_probability=0.3,
    elitism=True,
    n_jobs=-1,
    verbose=2,
    keep_top_k=8,
    criteria="max"
)

# Fit the genetic algorithm feature selector
selector.fit(X_train, np.ravel(y_train))

# Plot the evolution of the fitness
# plot_fitness_evolution(selector)

# Get the best features
best_features = selector.support_

# Transform the dataset to keep only the best features
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Train a model with the selected features
estimator.fit(X_train_selected, y_train)
accuracy = estimator.score(X_test_selected, y_test)

print("Selected features:", best_features)
print("Accuracy with selected features:", accuracy)


gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	20    	4204.2 	179.091    	4453.12    	3878.15    
1  	36    	4290.69	129.387    	4453.12    	3997.91    
2  	35    	4389.7 	134.588    	4642.84    	4123       
3  	36    	4391.65	143.31     	4563.04    	4123       
4  	36    	4392.12	144.905    	4563.04    	4063.02    
5  	37    	4420.69	145.591    	4587.73    	4148.1     
6  	39    	4415.69	171.711    	4627.65    	4067.98    
7  	38    	4437.96	147.256    	4627.65    	4047.99    
8  	39    	4498.94	103.146    	4627.65    	4222.98    
9  	37    	4456.35	179.48     	4627.65    	4008.15    
10 	36    	4520.79	168.594    	4627.65    	4038.09    
11 	37    	4561.5 	92.7278    	4627.65    	4317.95    
12 	37    	4510.56	175.309    	4667.96    	3942.85    
13 	32    	4499.83	193.702    	4667.96    	4023.04    
14 	35    	4566.01	133.078    	4667.96    	4048.16    
15 	36    	4587.74	109.781    	4637.75    	4142.95    
16 	38    	4605.23	111.079    	4697.7     	4142.95    
17 	34    

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GAFeatureSelectionCV was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
selector.support_

array([False, False, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False,  True, False,
       False,  True, False,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False])

In [ ]:
features_selected = [
    [False,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False],
    [False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False],
    [False, False, False, False, False, False, False,  True, False,
       False, False, False, False,  True, False, False,  True, False,
       False,  True, False,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False]
]

# Random Forest Most Important Features Selector

In [11]:
from collections import defaultdict

importances_dict = defaultdict(list)

params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 2, 3],
    'min_samples_split': [2, 5, 10]
}

all_importances = []
for n_estimators in params['n_estimators']:
  for max_depth in params['max_depth']:
    for min_samples_split in params['min_samples_split']:
      importances_dict = defaultdict(list)
      for i in range(3):
        model = RandomForestClassifier(random_state=i, n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
        model.fit(X_train, np.ravel(y_train))
      for j in range(len(model.feature_names_in_)):
        importances_dict[model.feature_names_in_[j]] = model.feature_importances_[j]
      mean_importances = pd.Series(importances_dict).groupby(level=0).mean()
      all_importances.append(mean_importances)
      print(f"Finished - n_estimators: {n_estimators}, max_depth: {max_depth}, min_samples_split: {min_samples_split}")

Finished - n_estimators: 50, max_depth: None, min_samples_split: 2
Finished - n_estimators: 50, max_depth: None, min_samples_split: 5
Finished - n_estimators: 50, max_depth: None, min_samples_split: 10
Finished - n_estimators: 50, max_depth: 2, min_samples_split: 2
Finished - n_estimators: 50, max_depth: 2, min_samples_split: 5
Finished - n_estimators: 50, max_depth: 2, min_samples_split: 10
Finished - n_estimators: 50, max_depth: 3, min_samples_split: 2
Finished - n_estimators: 50, max_depth: 3, min_samples_split: 5
Finished - n_estimators: 50, max_depth: 3, min_samples_split: 10
Finished - n_estimators: 100, max_depth: None, min_samples_split: 2
Finished - n_estimators: 100, max_depth: None, min_samples_split: 5
Finished - n_estimators: 100, max_depth: None, min_samples_split: 10
Finished - n_estimators: 100, max_depth: 2, min_samples_split: 2
Finished - n_estimators: 100, max_depth: 2, min_samples_split: 5
Finished - n_estimators: 100, max_depth: 2, min_samples_split: 10
Finished - 

[x1      0.015927
 x10     0.017882
 x101    0.042112
 x102    0.034067
 x103    0.041231
 x104    0.031301
 x105    0.031085
 x106    0.042342
 x132    0.019102
 x140    0.017967
 x149    0.017337
 x153    0.016563
 x156    0.016782
 x176    0.018234
 x191    0.017842
 x2      0.017476
 x22     0.018827
 x221    0.018617
 x229    0.019727
 x253    0.018052
 x286    0.018795
 x3      0.019098
 x304    0.017083
 x322    0.018566
 x323    0.017650
 x324    0.019173
 x329    0.017336
 x336    0.017705
 x35     0.016347
 x352    0.018720
 x36     0.018644
 x4      0.019787
 x40     0.019203
 x404    0.020049
 x413    0.016342
 x423    0.018630
 x459    0.016970
 x463    0.018928
 x499    0.018706
 x5      0.016791
 x58     0.017050
 x6      0.021857
 x65     0.016777
 x7      0.017816
 x74     0.017519
 x8      0.015848
 x81     0.016934
 x9      0.023360
 x99     0.015843
 dtype: float64,
 x1      0.015826
 x10     0.017720
 x101    0.044833
 x102    0.035061
 x103    0.041396
 x104    0.

In [27]:
best_features = [set(x.sort_values(ascending=False)[:7].keys()) for x in all_importances]
best_features

[{'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x105', 'x106', 'x6', 'x9'},
 {'x101', 'x102', 'x103', 'x105', 'x106', 'x6', 'x9'},
 {'x101', 'x102', 'x103', 'x105', 'x106', 'x6', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x9'},
 {'x101', 'x102', 'x103', 'x104', 'x1

In [28]:
whole_set = set()
for x in best_features:
  whole_set = whole_set.union(x)

whole_set

{'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x6', 'x9'}

In [35]:
from itertools import combinations

scores = []

for i in range(len(whole_set), 2, -1):
  for features in combinations(whole_set, i):
    X_train_tmp = X_train[list(features)]
    X_test_tmp = X_test[list(features)]
    model = RandomForestClassifier()
    model.fit(X_train_tmp, np.ravel(y_train))
    score = calculate_score(model, X_test_tmp, np.ravel(y_test))
    score_train = calculate_score(model, X_train_tmp, np.ravel(y_train))
    scores.append({'features': features, 'score_test': score, 'score_train': score_train})
    print(f"Features: {features}, score test: {score}, score train: {score_train}")

Features: ('x105', 'x9', 'x102', 'x6', 'x103', 'x106', 'x101', 'x104'), score test: 5133.466933867735, score train: 8400.0
Features: ('x105', 'x9', 'x102', 'x6', 'x103', 'x106', 'x101'), score test: 5233.266533066131, score train: 8600.0
Features: ('x105', 'x9', 'x102', 'x6', 'x103', 'x106', 'x104'), score test: 5253.306613226452, score train: 8600.0
Features: ('x105', 'x9', 'x102', 'x6', 'x103', 'x101', 'x104'), score test: 5213.226452905811, score train: 8600.0
Features: ('x105', 'x9', 'x102', 'x6', 'x106', 'x101', 'x104'), score test: 4912.625250501002, score train: 8600.0
Features: ('x105', 'x9', 'x102', 'x103', 'x106', 'x101', 'x104'), score test: 5313.426853707414, score train: 8600.0
Features: ('x105', 'x9', 'x6', 'x103', 'x106', 'x101', 'x104'), score test: 5333.466933867735, score train: 8600.0
Features: ('x105', 'x102', 'x6', 'x103', 'x106', 'x101', 'x104'), score test: 5313.426853707414, score train: 8600.0
Features: ('x9', 'x102', 'x6', 'x103', 'x106', 'x101', 'x104'), scor

In [42]:
pd.DataFrame(scores).sort_values(by='score_test', ascending=False).head(10)['features'].to_list()

[('x105', 'x102', 'x103', 'x106', 'x101'),
 ('x102', 'x103', 'x106', 'x101', 'x104'),
 ('x102', 'x106', 'x101', 'x104'),
 ('x102', 'x103', 'x106', 'x101'),
 ('x102', 'x106', 'x104'),
 ('x102', 'x103', 'x101', 'x104'),
 ('x105', 'x9', 'x103', 'x101', 'x104'),
 ('x102', 'x6', 'x103', 'x106', 'x104'),
 ('x105', 'x102', 'x103', 'x104'),
 ('x105', 'x9', 'x102', 'x103', 'x104')]